### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [32]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

#Easy to view Data for calculations- not needed, but helps me code/visualize 
school_data.head()
#student_data.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [33]:
# District Summary Calculations
#Total number of schools
district_total_schools = school_data.school_name.count()

#Total number of students
district_total_students = student_data.student_name.count()

#Total Budget
district_total_budget = school_data.budget.sum()

#Average Math Score
district_avg_math = student_data.math_score.mean()

#Average Reading Score
district_avg_rdg = student_data.reading_score.mean()

#Percent students passing math
total_passing_math = student_data.loc[student_data['math_score'] >= 70]['math_score'].count()
district_perc_passing_math = total_passing_math/district_total_students *100

#Percent students passing reading
total_passing_rdg = student_data.loc[student_data['reading_score'] >= 70]['reading_score'].count()
district_perc_passing_rdg = total_passing_rdg/district_total_students *100

#Overall Passing
#merge data for total passing rdg/math overall
school_student_data = pd.merge(student_data, school_data, how="left", on=["school_name"])

perc_overall_passing = len(school_student_data.loc[(school_student_data["math_score"] >= 70) & (school_student_data["reading_score"] >=70)])/district_total_students
district_overall_passing = perc_overall_passing*100

#Overall Summary
district_summary = pd.DataFrame({
    "Total Number of Schools": [district_total_schools],
    "Total Number of Students": [district_total_students],
    "Total Budget": [district_total_budget],
    "Average Math Score": [district_avg_math],
    "Average Reading Score": [district_avg_rdg],
    "Percent Students Passing Math": [district_perc_passing_math],
    "Percent Students Passing Reading": [district_perc_passing_rdg],
    "Overall Percent Passing": [district_overall_passing]
})

district_summary["Total Number of Schools"] = district_summary["Total Number of Schools"].map("{:,}".format)
district_summary["Total Number of Students"] = district_summary["Total Number of Students"].map("{:,}".format)
district_summary["Total Budget"] = (district_summary["Total Budget"]).map("${:,.2f}".format)
district_summary["Average Math Score"] = (district_summary["Average Math Score"]).map("{:.2f}".format)
district_summary["Average Reading Score"] = (district_summary["Average Reading Score"]).map("{:.2f}".format)
district_summary[ "Percent Students Passing Math"] = (district_summary[ "Percent Students Passing Math"]).map("{:.2f}".format)
district_summary[ "Percent Students Passing Reading"] = (district_summary[ "Percent Students Passing Reading"]).map("{:.2f}".format)
district_summary["Overall Percent Passing"] = (district_summary["Overall Percent Passing"]).map("{:.2f}".format)
district_summary.head()



,Total Number of Schools,Total Number of Students,Total Budget,Average Math Score,Average Reading Score,Percent Students Passing Math,Percent Students Passing Reading,Overall Percent Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,65.17


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [69]:
#School Summary
#merge data
school_student_data = pd.merge(student_data, school_data, how="left", on=["school_name"])

school_type = school_data.set_index(["school_name"])["type"]
num_schools = school_student_data["school_name"].value_counts()
school_budget = school_student_data.groupby(["school_name"]).mean()["budget"]
student_budget = school_budget/num_schools

#math
school_math_avg = school_student_data.groupby(["school_name"]).mean()["math_score"]

school_passing_math = school_student_data.loc[school_student_data["math_score"] >= 70] 
school_perc_math = school_passing_math.groupby(["school_name"]).count()["student_name"]/num_schools*100

#rdg
school_rdg_avg = school_student_data.groupby(["school_name"]).mean()["reading_score"]

school_passing_rdg = school_student_data.loc[school_student_data["reading_score"] >= 70] 
school_perc_rdg = school_passing_rdg.groupby(["school_name"]).count()["student_name"]/num_schools*100

#Overall Passing
per_passing_math=len(school_student_data.loc[school_student_data["math_score"]>=70])/district_total_students
per_passing_reading=len(school_student_data.loc[school_student_data["reading_score"]>=70])/district_total_students

#merge for overall passing per school
group_schools = school_student_data.set_index('school_name').groupby(['school_name'])
students_total_per_school = group_schools['Student ID'].count()
per_passing_overall = (school_student_data[(school_student_data['reading_score'] >= 70) & (school_student_data['math_score'] >= 70)].groupby('school_name')['Student ID'].count()/students_total_per_school)*100 


#Overall Summary
summary = pd.DataFrame({ 
            "School Type":school_type,
            "Total Students":num_schools,
            "School Budget":school_budget,
            "Student Budget": student_budget,
            "Average Math Score":school_math_avg,
            "Average Reading Score":school_rdg_avg,
            "Percent Passing Math":school_perc_math,
            "Percent Passing Reading":school_perc_rdg,
            "Percent Passing Overall":per_passing_overall})

#bonus - format
summary["Total Students"] = summary["Total Students"].map("{:,}".format)
summary["School Budget"] = summary["School Budget"].map("${:,.2f}".format)
summary["Student Budget"] = summary["Student Budget"].map("${:,.2f}".format)
summary["Average Math Score"] = summary["Average Math Score"].map("{:,.2f}".format)
summary["Average Reading Score"] = summary["Average Reading Score"].map("{:,.2f}".format)
summary["Percent Passing Math"] = summary["Percent Passing Math"].map("{:,.2f}".format)
summary["Percent Passing Reading"] = summary["Percent Passing Reading"].map("{:,.2f}".format)
summary["Percent Passing Overall"] = summary["Percent Passing Overall"].map("{:,.2f}".format)

summary

,School Type,Total Students,School Budget,Student Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Overall
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,54.29
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51,96.25,89.23
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,90.54


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [70]:
#Top Performing Schools
top_performing_schools = summary.sort_values(["Percent Passing Overall"], ascending = False)
top_performing_schools[:5]

,School Type,Total Students,School Budget,Student Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Overall
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.27,97.31,90.95
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87,96.54,90.58
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,90.54


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [71]:
#Lowsst Performing Schools
lowest_performing_schools = summary.sort_values(["Percent Passing Overall"], ascending = True)
lowest_performing_schools[:5]

,School Type,Total Students,School Budget,Student Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Overall
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.37,80.22,52.99
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [72]:
#Math Scores by Grade

#groupby
nine = school_student_data[school_student_data["grade"] == "9th"]
ten = school_student_data[school_student_data["grade"] == "10th"]
eleventh = school_student_data[school_student_data["grade"] == "11th"]
twelfth = school_student_data[school_student_data["grade"] == "12th"]

#math
math_nine = nine.groupby(["school_name"]).mean()["math_score"]
math_ten = ten.groupby(["school_name"]).mean()["math_score"]
math_eleventh = eleventh.groupby(["school_name"]).mean()["math_score"]
math_twelfth = twelfth.groupby(["school_name"]).mean()["math_score"]

#summary
math_per_grade_summary = pd.DataFrame({
    "9th Grade":math_nine,
    "10th Grade":math_ten,
    "11th Grade":math_eleventh,
    "12th Grade":math_twelfth})

#bonus format
math_per_grade_summary["9th Grade"] = math_per_grade_summary["9th Grade"].map("{:,.2f}".format)
math_per_grade_summary["10th Grade"] = math_per_grade_summary["10th Grade"].map("{:,.2f}".format)
math_per_grade_summary["11th Grade"] = math_per_grade_summary["11th Grade"].map("{:,.2f}".format)
math_per_grade_summary["12th Grade"] = math_per_grade_summary["12th Grade"].map("{:,.2f}".format)

math_per_grade_summary

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [73]:
#Rdg Scores by Grade

#groupby
nine = school_student_data[school_student_data["grade"] == "9th"]
ten = school_student_data[school_student_data["grade"] == "10th"]
eleventh = school_student_data[school_student_data["grade"] == "11th"]
twelfth = school_student_data[school_student_data["grade"] == "12th"]

#math
rdg_nine = nine.groupby(["school_name"]).mean()["reading_score"]
rdg_ten = ten.groupby(["school_name"]).mean()["reading_score"]
rdg_eleventh = eleventh.groupby(["school_name"]).mean()["reading_score"]
rdg_twelfth = twelfth.groupby(["school_name"]).mean()["reading_score"]

#summary
rdg_per_grade_summary = pd.DataFrame({
    "9th Grade":rdg_nine,
    "10th Grade":rdg_ten,
    "11th Grade":rdg_eleventh,
    "12th Grade":rdg_twelfth})

#bonus format
rdg_per_grade_summary["9th Grade"] = rdg_per_grade_summary["9th Grade"].map("{:,.2f}".format)
rdg_per_grade_summary["10th Grade"] = rdg_per_grade_summary["10th Grade"].map("{:,.2f}".format)
rdg_per_grade_summary["11th Grade"] = rdg_per_grade_summary["11th Grade"].map("{:,.2f}".format)
rdg_per_grade_summary["12th Grade"] = rdg_per_grade_summary["12th Grade"].map("{:,.2f}".format)

rdg_per_grade_summary

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [74]:
#Scores by School Spending
bins=[0,585,630,645,680]
bin_names=["<$585","$585-630","$630-645","$645-680"]

#Df & cut, change values
school_spending_orig = summary
school_spending_orig["Spending Range Per Student"] = pd.cut(student_budget, bins, labels = bin_names)
school_spending = school_spending_orig
school_spending["Average Math Score"] = school_spending["Average Math Score"].astype("float")
school_spending["Average Reading Score"] = school_spending["Average Reading Score"].astype("float")
school_spending["Percent Passing Math"] = school_spending["Percent Passing Math"].astype("float")
school_spending["Percent Passing Reading"] = school_spending["Percent Passing Reading"].astype("float")
school_spending["Percent Passing Overall"] = school_spending["Percent Passing Overall"].astype("float")

#Separate into bins
avg_math_school_spending = school_spending.groupby(["Spending Range Per Student"]).mean()["Average Math Score"]
avg_rdg_school_spending = school_spending.groupby(["Spending Range Per Student"]).mean()["Average Reading Score"]
perc_passing_math_school_spending = school_spending.groupby(["Spending Range Per Student"]).mean()["Percent Passing Math"]
perc_passing_rdg_school_spending = school_spending.groupby(["Spending Range Per Student"]).mean()["Percent Passing Reading"]
perc_overall_passing_school_spending = school_spending.groupby(["Spending Range Per Student"]).mean()["Percent Passing Overall"]


school_spending_summary = pd.DataFrame({
    "Average Math Score":avg_math_school_spending,
    "Average Reading Score":avg_rdg_school_spending,
    "Percent Passing Math":perc_passing_math_school_spending,
    "Percent Passing Reading":perc_passing_rdg_school_spending,
    "Percent Overall Passing":perc_overall_passing_school_spending})

#bonus format
school_spending_summary["Average Math Score"] = school_spending_summary["Average Math Score"].map("{:,.2f}".format)
school_spending_summary["Average Reading Score"] = school_spending_summary["Average Reading Score"].map("{:,.2f}".format)
school_spending_summary["Percent Passing Math"] = school_spending_summary["Percent Passing Math"].map("{:,.2f}".format)
school_spending_summary["Percent Passing Reading"] = school_spending_summary["Percent Passing Reading"].map("{:,.2f}".format)
school_spending_summary["Percent Overall Passing"] = school_spending_summary["Percent Overall Passing"].map("{:,.2f}".format)

school_spending_summary

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Overall Passing
Spending Range Per Student,,,,,
<$585,83.45,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [75]:
#Scores by School Size
size_bins=[0,1000,2000,5000]
size_bin_names=["Small(<1000)","Medium(1000-2000)","Large(2000-5000)"]

#Df & cut, change values
school_spending["Total Students"] = pd.cut(num_schools, size_bins, labels = size_bin_names)
school_size = school_spending
school_size["Average Math Score"] = school_size["Average Math Score"].astype("float")
school_size["Average Reading Score"] = school_size["Average Reading Score"].astype("float")
school_size["Percent Passing Math"] = school_size["Percent Passing Math"].astype("float")
school_size["Percent Passing Reading"] = school_size["Percent Passing Reading"].astype("float")
school_size["Percent Passing Overall"] = school_size["Percent Passing Overall"].astype("float")


#Separate into bins
avg_math_school_size = school_size.groupby(["Total Students"]).mean()["Average Math Score"]
avg_rdg_school_size = school_size.groupby(["Total Students"]).mean()["Average Reading Score"]
perc_passing_math_school_size = school_size.groupby(["Total Students"]).mean()["Percent Passing Math"]
perc_passing_rdg_school_size = school_size.groupby(["Total Students"]).mean()["Percent Passing Reading"]
perc_overall_passing_school_size = school_size.groupby(["Total Students"]).mean()["Percent Passing Overall"]


#Summary
school_size_summary = pd.DataFrame({
    "Average Math Score":avg_math_school_size,
    "Average Reading Score":avg_rdg_school_size,
    "Percent Passing Math":perc_passing_math_school_size,
    "Percent Passing Reading":perc_passing_rdg_school_size,
    "Percent Passing Overall":perc_overall_passing_school_size})

#bonus format
school_size_summary["Average Math Score"] = school_size_summary["Average Math Score"].map("{:,.2f}".format)
school_size_summary["Average Reading Score"] = school_size_summary["Average Reading Score"].map("{:,.2f}".format)
school_size_summary["Percent Passing Math"] = school_size_summary["Percent Passing Math"].map("{:,.2f}".format)
school_size_summary["Percent Passing Reading"] = school_size_summary["Percent Passing Reading"].map("{:,.2f}".format)
school_size_summary["Percent Passing Overall"] = school_size_summary["Percent Passing Overall"].map("{:,.2f}".format)

school_size_summary



,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Overall
Total Students,,,,,
Small(<1000),83.82,83.93,93.55,96.10,89.89
Medium(1000-2000),83.37,83.87,93.60,96.79,90.62
Large(2000-5000),77.75,81.34,69.96,82.77,58.28


## Scores by School Type

* Perform the same operations as above, based on school type

In [76]:
#Scores by School Type
avg_math_school_type = school_size.groupby(["School Type"]).mean()["Average Math Score"]
avg_rdg_school_type = school_size.groupby(["School Type"]).mean()["Average Reading Score"]
perc_passing_math_school_type = school_size.groupby(["School Type"]).mean()["Percent Passing Math"]
perc_passing_rdg_school_type = school_size.groupby(["School Type"]).mean()["Percent Passing Reading"]
perc_overall_passing_school_type = school_size.groupby(["School Type"]).mean()["Percent Passing Overall"]


#Summary
school_type_summary = pd.DataFrame({
    "Average Math Score":avg_math_school_type,
    "Average Reading Score":avg_rdg_school_type,
    "Percent Passing Math":perc_passing_math_school_type,
    "Percent Passing Reading":perc_passing_rdg_school_type,
    "Percent Passing Overall":perc_overall_passing_school_type})

#bonus format
school_type_summary["Average Math Score"] = school_type_summary["Average Math Score"].map("{:,.2f}".format)
school_type_summary["Average Reading Score"] = school_type_summary["Average Reading Score"].map("{:,.2f}".format)
school_type_summary["Percent Passing Math"] = school_type_summary["Percent Passing Math"].map("{:,.2f}".format)
school_type_summary["Percent Passing Reading"] = school_type_summary["Percent Passing Reading"].map("{:,.2f}".format)
school_type_summary["Percent Passing Overall"] = school_type_summary["Percent Passing Overall"].map("{:,.2f}".format)

school_type_summary

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Overall
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67


## Observations from Data Analysis

####### The top 5 schools are all charter schools, while the bottom 5 schools are all district schools. Charter schools have high average scores, and therefore a higher percent of students passing overall (90.43% for charter, 53.67% for district).
####### In general, small schools (less than 1,000 students) and medium schools (1,000-2,000 students) have higher scores than large schools (2,000-5,000 students).
####### Reading and math scores per grade are relatively consistent per school.